In [ ]:
%cd ..
%config Completer.use_jedi = False
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import tensorflow as tf
import tensorflow_datasets as tfds

import tensorflow_datasets_bw as datasets
from tensorflow_datasets_bw import visualize
import dppp

In [ ]:
scale_factor = 5
resize_method = tf.image.ResizeMethod.BICUBIC

builder_kwargs = {
    'resize_method': resize_method,
    'scale': scale_factor,
    'antialias': True
}
images = tfds.load(name="set5", split="test", builder_kwargs=builder_kwargs) \
                .map(datasets.map_on_dict(datasets.to_float32)) \
                .map(datasets.map_on_dict(datasets.from_255_to_1_range))

In [ ]:
# Pick a new random image
image_dict = datasets.get_one_example(images, random=True)
image_hr = image_dict['hr'][None, ...]
image_lr = image_dict['lr'][None, ...]

In [ ]:
# Load the denoiser
denoiser, (_, _) = dppp.load_denoiser('models/drunet+_0.0-0.2.h5')

In [ ]:
denoiser_stddevs = [0.02, 0.05, 0.1, 0.2]
num_steps = 100

# Print the PSNR every 20th step
callbacks = [dppp.callback_print_psnr('psnr', 20, image_hr)]

reconstructed = []

for dstddev in denoiser_stddevs:
    print()
    print(f"Running for denoiser stddev {dstddev:0.2f}:")
    print("=================================")
    reconstructed.append(dppp.dmsp_super_resolve(image_lr, scale_factor,
                                                 denoiser, dstddev,
                                                 num_steps=num_steps, callbacks=callbacks))
    

print()
print("FINAL RESULTS:")
for dstddev, rec in zip(denoiser_stddevs, reconstructed):
    psnr = dppp.psnr(image_hr, rec).numpy()[0]
    print(f"PSNR for denoiser stddev {dstddev:0.2f}: {psnr}")